In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import requests

In [2]:
import urllib.request
import urllib

In [3]:
def qcewCreateDataRows(csv):
    dataRows = []
    try: dataLines = csv.decode().split('\r\n')
    except er: dataLines = csv.split('\r\n');
    for row in dataLines:
        dataRows.append(row.split(','))
    return dataRows

In [4]:
def qcewGetAreaData(year,qtr,area):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[QTR]",qtr.lower())
    urlPath = urlPath.replace("[AREA]",area.upper())
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)

In [5]:
def qcewGetIndustryData(year,qtr,industry):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/industry/[IND].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[QTR]",qtr.lower())
    urlPath = urlPath.replace("[IND]",industry)
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)

In [6]:
def qcewGetSizeData(year,size):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/1/size/[SIZE].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[SIZE]",size)
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)

In [7]:
Michigan_Data = qcewGetAreaData("2015","1","26000")
Auto_Manufacturing = qcewGetIndustryData("2015","1","3361")
SizeData = qcewGetSizeData("2015","6")

In [8]:
print(Michigan_Data[5][2])

"1022"


In [9]:
# prints the area_fips in row 1.
# remember row zero contains field names
print(Auto_Manufacturing[1][0])

"01000"


In [10]:
# prints the own_code in row 1.
# remember row zero contains field names
print(SizeData[1][1])

"5"


In [11]:
pip install -U blsconnect

In [12]:
api_key = 'd7c0ebba465a4848afbcfb17a9238452'

In [13]:
from blsconnect import RequestBLS, bls_search

MY_API_KEY = 'd7c0ebba465a4848afbcfb17a9238452'
bls = RequestBLS(key=MY_API_KEY)

# Get seasonally-adjusted unemployment rates for Florida, Georgia, and all U.S.
series_names = bls_search(data="U3",
                          state=["FL", "GA", "NY", "NJ", "US"],
                          sa=True)

df = bls.series(series_names,
                start_year=2010,
                end_year=2019)

df.head()

,year,period,periodName,LASST120000000000003,LASST130000000000003,LASST360000000000003,LASST340000000000003,LNS14000000
0,2010,M01,January,11.3,10.5,8.9,9.8,9.8
1,2010,M02,February,11.2,10.6,8.8,9.7,9.8
2,2010,M03,March,11.2,10.6,8.8,9.7,9.9
3,2010,M04,April,11.1,10.5,8.7,9.6,9.9
4,2010,M05,May,11.0,10.4,8.6,9.5,9.6


In [14]:
df.rename(columns={"LASST120000000000003": "Florida"})

,year,period,periodName,Florida,LASST130000000000003,LASST360000000000003,LASST340000000000003,LNS14000000
0,2010,M01,January,11.3,10.5,8.9,9.8,9.8
1,2010,M02,February,11.2,10.6,8.8,9.7,9.8
2,2010,M03,March,11.2,10.6,8.8,9.7,9.9
3,2010,M04,April,11.1,10.5,8.7,9.6,9.9
4,2010,M05,May,11.0,10.4,8.6,9.5,9.6
...,...,...,...,...,...,...,...,...
115,2019,M08,August,3.0,3.3,3.9,3.5,3.7
116,2019,M09,September,2.9,3.2,3.9,3.6,3.5
117,2019,M10,October,2.9,3.2,3.9,3.7,3.6
118,2019,M11,November,2.8,3.1,3.9,3.7,3.5


In [15]:
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2014"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()

ModuleNotFoundError: No module named 'prettytable'